이번에는 간단한 특징 벡터를 구성해보고, 함께 나타나는 동시발생 단어들의 정보를 활용하는 법을 살펴봅니다.

### TF 행렬 만들기

앞서 TF-IDF에 대해서 살펴보았습니다. TF-IDF뿐만 아니라 단어의 문서별 출현 횟수를 가르키는 TF 또한 훌륭한 특징이 될 수 있습니다. 예를 들어 어떤 단어가 문서마다 출현한 횟수가 차원별로 구성되면, 하나의 특징 벡터를 이룰수 있습니다.

In [1]:
import pandas as pd

def get_term_frequency(document, word_dict = None):
    if word_dict is None:
        word_dict = {}
    words = document.split()
    
    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])
        
    return pd.Series(word_dict).sort_values(ascending = False)

In [2]:
doc1 = '''
지능 지수 라는 말 들 어 보 셨 을 겁니다 . 여러분 의 지성 을 일컫 는 말 이 죠 . 그런데 심리 지수 란 건 뭘까요 ? 사람 들 이 특정 한 식 으로 행동 하 는 이유 에 대해 여러분 은 얼마나 알 고 계시 나요 ? 또 타인 이나 심지어 여러분 의 행동 을 예측 하 는 일 은 얼마나 잘 하 시 나요 ? 또 , 심리학 에 대해 갖춘 지식 중 에서 어느 정도 나 잘못 된 것 일까요 ? 심리학 에 관한 열 가지 신화 를 통해 잘못 된 것 들 을 알아보 도록 하 죠 . 여러분 은 한 번 쯤 들 어 보 셨 을 법 한 것 은 자신 들 의 심리학 에 대해 고려 할 때 , 거의 항상 남자 는 화성 에서 왔 고 , 여자 는 금성 에서 온 것 같 다고 합니다 . 하지만 실제로 남자 와 여자 는 얼마나 다른 걸까요 ? 이 를 알아보 기 위해 , 일단 남녀 사이 에 확실 하 게 차이 나 는 것 을 살펴보 고 심리학 적 인 성별 간 의 차이점 을 동일 한 척도 상 에서 대비 해 보 도록 하 겠 습니다 . 남자 와 여자 간 에 실제로 차이 나 는 능력 중 하나 는 그 들 이 공 을 얼마나 멀리 던질 수 있 느냐 하 는 것 입니다 . 여기 남자 들 의 데 이타 를 보 시 면 , 정상 분포 곡선 이 라는 걸 볼 수 있 습니다 . 남자 들 소수 는 정말 멀리 던지 고 , 남자 들 소수 는 멀리 던지 지 못하 지만 , 남자 들 대부분 은 평균 적 인 거리 를 던졌 습니다 . 여자 들 도 역시 비슷 한 분포 상태 를 보입니다 만 사실 남녀 사이 엔 커다란 차이 가 있 습니다 . 사실 , 평균 수준 의 남자 라면 모든 여성 중 대략 98 % 보다 더 멀리 던질 수 있 거든요 . 이 와 동일 하 게 표준 화 된 척도 상 에서 심리학 에서 말 하 는 성별 간 의 차이 를 살펴 봅시다 . 심리학자 라는 여러분 에게 말 하 길 남자 들 의 공간 지각 능력 이 여자 들 보다 뛰어나 다고 할 겁니다 . 예 를 들 어 , 지도 읽 는 능력 같 은 건데 , 맞 는 말 입니다 . 하지만 그 차이 의 정도 를 살펴봅시다 . 아주 작 죠 . 두 선 이 너무 근접 해서 거의 겹칠 정도 입니다 .
'''

doc2 = '''
최상 의 제시 유형 은 학습 자 에 좌우 되 는 것 이 아니 라 학습 해야 할 내용 에 따라 좌우 됩니다 . 예 를 들 어 여러분 이 운전 하 기 를 배울 때 실제로 몸 으로 체감 하 는 경험 없이 누군가 가 어떻게 할 지 이야기 하 는 것 을 듣 는 것 만 으로 배울 수 있 습니까 ? 연립 방정식 을 풀 어야 하 는데 종이 에 쓰 지 않 고 머리 속 에서 말 하 는 것 으로 풀 수 가 있 을까요 ? 또는 만일 여러분 이 체감 형식 의 학습 자 유형 이 라면 , 건축학 시험 을 해석 적 춤 을 이용 하 여 수정 할 수 있 을까요 ? 아니 죠 ! 배워야 할 내용 을 제시 된 유형 에 맞추 어야 합니다 , 당신 에게 맞추 는 게 아니 라요 . 여러분 들 상당수 가 " A " 급 의 우등 생 이 라는 걸 아 는데 , 조만간 중등 학력 인증 시험 ( GCSE ) 결과 를 받 게 되 시 겠 네요 . 그런데 , 만일 , 여러분 들 이 희망 했 던 성적 을 받 지 못하 게 된다 해도 여러분 들 의 학습 방식 을 탓 해서 는 안 되 는 겁니다 . 여러분 이 비난 할 수 있 는 한 가지 는 바로 유전자 입니다 . 이건 최근 에 런던 대학교 ( UCL ) 에서 수행 했 던 연구 결과 는 여러 학생 들 과 그 들 의 중등 학력 인증 시험 결과 사이 의 차이 중 58 % 는 유전 적 인 요인 으로 좁혀졌 습니다 . 매우 정밀 한 수치 처럼 들립니다 . 그러면 어떻게 알 수 있 을까요 ? 유전 적 요인 과 환경 적 요인 의 상대 적 기여 도 를 알 고 싶 을 때 우리 가 사용 할 수 있 는 방식 은 바로 쌍둥이 연구 입니다 . 일 란 성 쌍생아 의 경우 환경 적 요인 과 유전 적 요인 모두 를 100 % 똑같이 공유 하 게 되 지만 이란 성 쌍생아 의 경우 는 100 % 동일 한 환경 을 공유 하 지만 유전자 의 경우 여타 의 형제자매 들 처럼 50 % 만 공유 하 게 됩니다 . 따라서 일 란 성 쌍둥이 와 이란 성 쌍둥이 사이 의 인증 시험 결과 가 얼마나 비슷 한지 비교 해 보 고 여기 에 약간 의 수학 적 계산 을 더하 게 되 면 그 수행 능력 의 차이 중 어느 정도 가 환경 적 요인 의 탓 이 고 어느 정도 가 유전자 탓 인지 를 알 수 있 게 됩니다 .
'''

doc3 = '''
그러나 이 이야기 는 세 가지 이유 로 인해 신화 입니다 . 첫째 , 가장 중요 한 건 실험실 가운 은 흰색 이 아니 라 회색 이 었 다 라는 점 이 죠 . 둘째 , 참 여자 들 은 실험 하 기 전 에 와 참여 자 들 이 걱정 을 표현 할 때 마다 상기 시키 는 말 을 들 었 는데 , 전기 충격 이 고통 스럽 기 는 하 지만 , 치명 적 이 지 는 않 으며 실제로 영구 적 인 손상 을 남기 는 일 은 없 을 거 라는 것 이 었 습니다 . 셋째 , 참 여자 들 은 단지 가운 을 입 은 사람 이 시켜 전기 충격 을 주지 는 않 았 죠 . 실험 이 끝나 고 그 들 의 인터뷰 를 했 을 때 모든 참여 자 들 은 강한 신념 을 밝혔 는데 , ' 학습 과 처벌 ' 연구 가 과학 적 으로 가치 있 는 목적 을 수행 했 기 때문 에 비록 동료 참여 자 들 에게 가해진 순간 적 인 불편 함 에 반해서 과학 을 위해서 오래 남 을 성과 를 얻 을 것 이 라고 말 이 죠 . 그러 다 보 니 제 가 이야기 를 한 지 벌써 12 분 이 되 었 습니다 . 여러분 들 중 에 는 아마 거기 앉 아서 제 이야기 를 들으시는 동안 저 의 말투 와 몸짓 을 분석 하 면서 제 가 말 하 는 어떤 것 을 인지 해야 할까 해결 하 려고 하 셨 을 겁니다 , 제 가 진실 을 이야기 하 는 지 , 또는 거짓말 을 하 고 있 는 것 인지 말 이 죠 . 만일 그러 셨 다면 , 아마 지금 쯤 완전히 실패 하 셨 을 겁니다 . 왜냐하면 우리 모두 가 사람 이 말 하 는 패턴 과 몸짓 으로 도 거짓말 여부 를 알아내 는 것 이 가능 하 다고 생각 하 지만 , 오랜 세월 수백 회 에 걸쳐 행해진 실제 심리 검사 의 결과 를 보 면 우리 들 모두 는 , 심지어 경찰관 이나 탐정 들 을 포함 해서 도 기본 적 으로 몸짓 과 언어 적 패턴 으로 거짓말 을 탐지 하 는 것 은 운 에 맞 길 수 밖 에 는 없 는 것 입니다 . 흥미 롭 게 도 한 가지 예외 가 있 는데요 : 실종 된 친척 을 찾 아 달 라고 호소 하 는 TV 홍보 입니다 .
'''

In [5]:
def get_tf(docs):
    vocab = {}
    tfs = []
    
    for d in docs:
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]

    from operator import itemgetter
    import numpy as np

    stats = []
    for word, freq in vocab.items():
        tf_v = []
        
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tf_v += [tfs[idx][word]]
                
            else:
                tf_v += [0]
                
        stats.append((word, freq, *tf_v))
    
    return pd.DataFrame(stats, columns=('word',
                                        'frequency',
                                        'doc1',
                                        'doc2',
                                        'doc3')).sort_values('frequency', ascending=False)

In [12]:
get_tf([doc1, doc2, doc3])

,word,frequency,doc1,doc2,doc3
0,는,47,15,14,18
1,을,39,8,10,21
2,.,36,16,10,10
3,하,33,10,9,14
4,이,32,8,8,16
...,...,...,...,...,...
273,항상,1,1,0,0
274,반해서,1,0,0,1
275,위해서,1,0,0,1
276,오래,1,0,0,1


마지막 3개 열에 해당되는 $TF(w,d_1), TF(w,d_2), TF(w,d_3)$ 는 문서에 대한 단어별 출현 횟수를 활용한 특징 벡터가 될 것입니다. 예를 들어 "여러분"이라는 단어는 [5, 6, 1]이라는 특징 벡터를 갖습니다.

하지만 문서가 지나치게 많다면 벡터의 차원 역시 지나치게 커질 수 있습니다. 문서가 10,000개가 있다고 한다면 단어당 10,000 차원의 벡터가 만들어집니다. 문제는 이 10,000 차원의 벡터 대부분의 값이 0으로 채워진다는 것입니다. 이렇게 벡터의 극히 일부만 의미 있는 값으로 채워진 벡터가 **희소 벡터**입니다. 희소 벡터의 각 차원은 대부분의 경우 0이므로 유의미한 특정 통계를 없는데 큰 걸림돌이 될 수 있습니다.

### 컨텍스트 윈도우로 함께 출현한 단어들의 정보 활용하기

함께 나타나는 $동시발생^{co-occurence}$ 단어들을 활용한 방법입니다. 앞에서 가정한 대로, 의미가 비슷한 단어라면 쓰임새 역시 비슷할 것입니다. 쓰임새가 비슷하기에 비슷한 문장안에서 비슷한 역할로 사용될 것이고, 따라서 함게 나타나는 단어들이 서로 유사할 것입니다. 이러한 관점에서 함께 나타나는 "서로 유사한" 단어들이 유사한 형태의 벡터를 갖게 해야합니다.

함께 나타나는 단어들을 조사하기 위해 $윈도윙^{windowing}$ 실행합니다. 윈도잉이랑 윈도우를 움직이며 그 안에 있는 유닛들의 정보를 취합하는 방법을 이르며 이때 사용되는 윈도우를 **$컨텍스트 윈도우^{context window}$**라고 합니다. 이 방법은 단어별로 윈도우 내에 속해있는 이웃 단어들의 출현 빈도를 세어 행렬로 나타내는 것입니다.

"윈도우 크기"라는 하나의 하이퍼파라미터가 추가되므로 사용자가 그 값을 정해주어야 합니다. 만약 윈도우 크기가 너무 크다면 현재 단어와 관계없는 단어들까지 출현 빈도를 셀 수 있습니다. 따라서 적절한 윈도우 크기를 정하는 것이 중요합니다.

파이썬 코드를 통해 미리 분절 처리한 만개의 예제 문장들에 대해 윈도잉을 수행할 수 있습니다.

<br></br>
![](./images/5-6-2-cooccurrence.jpg)
<br></br>

문장들을 입력으로 받아 주어진 윈도우 크기 w_size 내에서 함께 출현한 단어들의 빈도를 세는 함수는 다음과 같습니다.

In [13]:
from collections import defaultdict

import pandas as pd

def get_context_counts(lines, w_size = 2):
    co_dict = defaultdict(int)
    
    for line in lines:
        words = line.spilt()
        
        for i, w in enumerate(words):
            for c in words[i - w_size: i + w_size]:
                if w != c:
                    co_dict[(w,c)] += 1
                    
    return pd.Series(co_dict)

앞서 TF-IDF를 위해 작성했던 get_term_frequency() 함수를 활용하여 동시발생 정보를 통해 벡터를 만듭니다.

In [14]:
def co_occurrence(co_dict, vocab):
    data = []
    
    for word1 in vocab:
        row = []
        
        for word2 in vocab:
            try:
                count = co_dict[(word1, word2)]
            
            except KeyError:
                count = 0
                
            row.append(count)
        
        data.append(row)
    
    return pd.DataFrame(data, index = vocab, columns = vocab)

그리고 이 코드를 통해 얻은 결과의 일부는 다음과 같습니다. 결과에 따르면 1000개의 문서에서는 "습니다"의 컨텍스트 윈도우 내에 쉼표가 470번 등장합니다.

<br></br>
![](./images/5-6-2-cooccurrence2.jpg)
<br></br>

왼쪽의 동시출현 빈도가 높은 단어들은 대부분 값이 잘 채워져있습니다. 하지만 오른쪽의 동시 출현 빈도가 낮은 단어들은 많은 부분이 0으로 채워져있습니다. 단어 자체의 출력 빈도가 낮은 경우에는 그 문제가 더욱 심각합니다. 이런 희소 벡터들이 많으면, 유사도를 구하거나 벡터간 연산을 할때 직교하는 경우가 많아 매우 곤란합니다.

<br></br>
![](./images/5-6-2-cooccurrence3.jpg)
<br></br>

앞의 컨텍스트 윈도우 실행 결과로 얻은 특징 벡터들을 t-SNE를 이용하여 시각화한 모습은 다음과 같습니다. 비슷한 단어끼리 모이지 않은 것도 많지만 비슷한 단어끼리 붙어 있는 경우도 종종 볼 수 있습니다. 하지만 여전히 앞의 벡터를 특징 벡터로 활용하기엔 뭔가 부족합니다.

<br></br>
![](./images/5-6-2-tsne.jpg)
<br></br>